In [6]:
from NCP.utils import frnp

import torch
from torch.optim import Adam
from NCP.model import NCPOperator, NCPModule
from NCP.nn.layers import MLP
from NCP.nn.losses import CMELoss
from NCP.nn.nf_module import NFModule
from NCP.utils import frnp, FastTensorDataLoader
from NCP.cdf import get_cdf, quantile_regression, get_mean_from_nf, compute_marginal, get_cdf_from_nf
import lightning as L
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint
import numpy as np
from NCP.metrics import smooth_cdf
from NCP.cdf import compute_marginal
from NCP.examples.tools.lincde import lincde

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm

import normflows as nf
from sklearn.preprocessing import StandardScaler
from uci_datasets import Dataset
from conditionalconformal import CondConf
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

In [7]:
xscaler = StandardScaler()
yscaler = StandardScaler()

data = Dataset('houseelectric')

X_train, Y_train, X_test, Y_test = data.get_split(split=0)
X_train, X_val, Y_train, Y_val = train_test_split(X_train, Y_train, test_size=0.1, random_state=0)

# subsampling for computation reasons:
X_train = X_train[:50000]
Y_train = Y_train[:50000].reshape(-1, 1)
X_val = X_val[:1000]
Y_val = Y_val[:1000].reshape(-1, 1)
X_test = X_test[:200]
Y_test = Y_test[:200].reshape(-1,1)

X_train = xscaler.fit_transform(X_train)
Y_train = yscaler.fit_transform(Y_train)

X_val = xscaler.transform(X_val)
Y_val = yscaler.transform(Y_val)
X_test = xscaler.transform(X_test)
Y_test = yscaler.transform(Y_test)

X_train_torch = frnp(X_train)
Y_train_torch = frnp(Y_train)
X_val_torch = frnp(X_val)
Y_val_torch = frnp(Y_val)
X_test_torch = frnp(X_test)
Y_tes_torch = frnp(Y_test)

#y discretisation for computing cdf
spread = np.max(Y_train) - np.min(Y_train)
p1, p99 = np.min(Y_train), np.max(Y_train)
y_discr, step = np.linspace(p1-0.1*spread, p99+0.1*spread, num=5000, retstep=True)
y_discr_torch = torch.Tensor(y_discr.reshape((-1, 1)))

k_pdf = compute_marginal(bandwidth='scott').fit(Y_train)
marginal = lambda x : torch.Tensor(np.exp(k_pdf.score_samples(x.reshape(-1, 1))))

train_dl = FastTensorDataLoader(X_train_torch, Y_train_torch, batch_size=len(X_train_torch), shuffle=False)
val_dl = FastTensorDataLoader(X_val_torch, Y_val_torch, batch_size=len(X_val_torch), shuffle=False)

alpha = 0.1
NEXP=1

houseelectric dataset, N=2049280, d=11


In [16]:
output_shape=500

device = 'cuda' if torch.cuda.is_available() else 'cpu'

class CustomModelCheckpoint(ModelCheckpoint):
    def on_save_checkpoint(self, trainer, pl_module, checkpoint):
        X, Y = trainer.model.batch
        trainer.model.model._compute_data_statistics(X, Y)

def restore_buffers_shape(model, state_dict):
    model._sing_val = torch.zeros_like(state_dict['model._sing_val']).to('cpu')
    model._sing_vec_l = torch.zeros_like(state_dict['model._sing_vec_l']).to('cpu')
    model._sing_vec_r = torch.zeros_like(state_dict['model._sing_vec_r']).to('cpu')

MLP_kwargs_U = {
    'input_shape': X_train.shape[-1],
    'output_shape': output_shape,
    'n_hidden': 4,
    'layer_size': 128,
    'dropout': 0.,
    'iterative_whitening': False,
    'activation': torch.nn.ReLU
}

MLP_kwargs_V = {
    'input_shape': Y_train.shape[-1],
    'output_shape': output_shape,
    'n_hidden': 4,
    'layer_size':128,
    'dropout': 0,
    'iterative_whitening': False,
    'activation': torch.nn.ReLU
}

loss_fn = CMELoss
loss_kwargs = {
    'mode': 'split',
    'gamma': 2e-1}

reg = NCPOperator(U_operator=MLP, V_operator=MLP, U_operator_kwargs=MLP_kwargs_U, V_operator_kwargs=MLP_kwargs_V)
optimizer_kwargs = {
        'lr': 1e-3
        }
NCP_module = NCPModule(
    reg,
    Adam,
    optimizer_kwargs,
    CMELoss,
    loss_kwargs
)

early_stop = EarlyStopping(monitor="val_loss", patience=300, mode="min")
checkpoint_callback = CustomModelCheckpoint(save_top_k=1, monitor="val_loss", mode="min")

trainer = L.Trainer(**{
    'accelerator': device,
    'max_epochs': int(5e3),
    'log_every_n_steps': 1,
    'enable_progress_bar': True,
    'devices': 1,
    'enable_checkpointing': True,
    'num_sanity_val_steps': 0,
    'enable_model_summary': False,
    }, callbacks=[early_stop, checkpoint_callback])

trainer.fit(NCP_module, train_dataloaders=train_dl, val_dataloaders=val_dl)

# recover best model during training
best_model_dict = torch.load(checkpoint_callback.best_model_path)
restore_buffers_shape(reg, best_model_dict['state_dict'])
NCP_module.load_state_dict(best_model_dict['state_dict'])
best_model = NCP_module.model


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 1237: 100%|██████████| 1/1 [00:00<00:00,  4.45it/s, v_num=130, val_loss=2.400, train_loss=-0.0698]  


In [10]:
best_models_nf = {}

for exp in range(1):
    L.seed_everything(exp)

    base = nf.distributions.base.DiagGaussian(1)

    # Define list of flows (2 flows to emulate our two MLP approach, each with more capacity than our MLPs)
    num_flows = 2
    latent_size = 1
    hidden_units = 128
    num_blocks = 3
    flows = []
    for i in range(num_flows):
        flows += [nf.flows.AutoregressiveRationalQuadraticSpline(latent_size, num_blocks, hidden_units, 
                                                    num_context_channels=X_train.shape[-1])]
        flows += [nf.flows.LULinearPermute(latent_size)]

    # If the target density is not given
    model = nf.ConditionalNormalizingFlow(base, flows)

    nf_module = NFModule(model,
        Adam,
        optimizer_kwargs)

    checkpoint_callback_vanilla = ModelCheckpoint(monitor="val_loss", mode="min")

    trainer = L.Trainer(**{
        'accelerator': device,
        'max_epochs': int(5e3),
        'log_every_n_steps': 1,
        'enable_progress_bar': True,
        'devices': 1,
        'enable_checkpointing': True,
        'num_sanity_val_steps': 0,
        'enable_model_summary': False,
        }, callbacks=[checkpoint_callback_vanilla])

    trainer.fit(nf_module, train_dataloaders=train_dl, val_dataloaders=val_dl)

    checkpoint_callback_vanilla.best_model_path
    best_model_dict = torch.load(checkpoint_callback_vanilla.best_model_path)
    nf_module.load_state_dict(best_model_dict['state_dict'])

    best_models_nf[exp] = nf_module.model

Seed set to 0
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Epoch 4999: 100%|██████████| 1/1 [00:00<00:00,  4.33it/s, v_num=123, val_loss=-2.08, train_loss=-2.27]  

`Trainer.fit` stopped: `max_epochs=5000` reached.


Epoch 4999: 100%|██████████| 1/1 [00:00<00:00,  4.21it/s, v_num=123, val_loss=-2.08, train_loss=-2.27]


In [17]:
# plots with quantiles for one model

pred_test = best_model.conditional_expectation(X_test, Y_train, postprocess='whitening').reshape(-1, 1)
nf_pred = np.zeros(X_test.shape[0])
nf_quantiles = np.zeros((X_test.shape[0], 2))
pred_quantiles = np.zeros((X_test.shape[0], 2))
for i, xi in enumerate(tqdm(X_test)):
    nf_pred[i] = get_mean_from_nf(best_models_nf[0], torch.Tensor([xi]), 1000)
    nf_quantiles[i] = quantile_regression(best_models_nf[0], torch.Tensor([xi]),
                                          y_discr_torch, alpha=alpha, 
                                          postprocess='whitening', marginal=marginal, 
                                          model_type='NF')
    pred_quantiles[i] = quantile_regression(best_model, torch.Tensor([xi]),
                                            y_discr_torch, alpha = alpha, 
                                            postprocess='whitening', marginal=marginal, 
                                            model_type='NCP')
nf_pred= nf_pred.reshape(-1,1)

100%|██████████| 200/200 [39:36<00:00, 11.88s/it]


In [18]:
from sklearn.ensemble import RandomForestRegressor
from deel.puncc.regression import SplitCP
# split conformal with random forests

trained_linear_model = RandomForestRegressor().fit(X_train, Y_train.flatten())

# Instanciate the split conformal wrapper for the linear model.
# Train argument is set to False because we do not want to retrain the model
split_cp = SplitCP(trained_linear_model, train=False)

# With a calibration dataset, compute (and store) nonconformity scores
split_cp.fit(X_fit=X_train, y_fit=Y_train.flatten(), X_calib=X_val, y_calib=Y_val.flatten())

# Obtain the model's point prediction y_pred and prediction interval
# PI = [y_pred_lower, y_pred_upper] for a target coverage of 90% (1-alpha).
y_pred, y_pred_lower, y_pred_upper = split_cp.predict(X_test, alpha=alpha)
quants = np.array([y_pred_lower, y_pred_upper]).T

In [20]:
print(np.mean((Y_test.flatten() >= pred_quantiles[:,0]) *(Y_test.flatten() <= pred_quantiles[:,1])))
print(np.mean((Y_test.flatten() >= nf_quantiles[:,0]) *(Y_test.flatten() <= nf_quantiles[:,1])))
print(np.mean((Y_test.flatten() >= quants[:,0]) *(Y_test.flatten() <= quants[:,1])))

0.96
0.885
0.855


In [21]:
print(np.mean(pred_quantiles[:,1] - pred_quantiles[:,0]), np.std(pred_quantiles[:,1] - pred_quantiles[:,0]))
print(np.mean(nf_quantiles[:,1] - nf_quantiles[:,0]), np.std(nf_quantiles[:,1] - nf_quantiles[:,0]))
print(np.mean(quants[:,1] - quants[:,0]), np.std(quants[:,1] - quants[:,0]))

1.7996339404582977 0.7520310413554643
0.11872025499338633 0.07720246278158797
0.11789847349119817 9.53182800550177e-17


In [ ]:
improt matplotlib.pyplot as plt

plt.scatter(X_test, Y_test)
